In [2]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import json
import pickle

In [ ]:
# base_url = 'https://api.mobygames.com/v1/'
# endpoint_var = 'games'
# api_key = open("../data/mobygames_api_key.txt")
# endpoint= base_url + endpoint_var

In [ ]:
# parameters = {"format":"normal", 
#               "api_key": api_key, 
#               "genre":158,
#               "offset":99
# #               "id":2
#              }

In [ ]:
# response = requests.get(endpoint, params=parameters)
# response

In [ ]:
# res = response.json()

In [ ]:
# type(res)

In [ ]:
# len(res["games"])

In [ ]:
# res["games"][0].keys()

In [ ]:
# print(res["games"][0]["moby_url"])

In [ ]:
#this returns one of the first release dates from one of the platforms - this is one way to get dates
# print(res["games"][0]["platforms"][0]["first_release_date"])
# print(res["games"][0]["platforms"][0]["platform_id"])

In [ ]:
# this gets to a json file that actually has the companies and their roles 
# https://api.mobygames.com/v1/games/2/platforms/2?api_key=MYKEY

### A function for looping through responses of 100, offset by 100, until it hits the end
the goal of this funciton is to fill up the data dictionary with a bunch of other response dictionaries that contain all games for the given "genre" - in this case, 158, or "vehicle simulator"

In [75]:
y=0
x=1
data = {}

In [76]:
# WARNING: THIS DOESN'T BREAK - RESPONSE WILL BE 200 EVEN IF OFFSET IS TOO HIGH, APPARENTLY
# NEED TO CHECK THE RES["GAMES"] DICT TO SEE IF EMPTY, THATS THE WAY TO DO IT - use len like this:
# len(data["games12"])

#FIXED problem described in comments above

while True:
        base_url = 'https://api.mobygames.com/v1/'
        endpoint_var = 'games'
        api_key = open("../data/mobygames_api_key.txt")
        endpoint= base_url + endpoint_var
        parameters = {"format":"normal", 
                  "api_key": api_key, 
                  "genre":158,
                  "offset":y}
        response = requests.get(endpoint, params=parameters)
        print(response)
        res = response.json()
        print(len(res["games"]))
        if len(res["games"]) > 0:
            res["games"+str(x)] = res["games"]
            del res["games"]
            data = {**data, **res}
            y+=100
            x+=1
            sleep(10.1)
        else:
            break

<Response [200]>
100
<Response [200]>
100
<Response [200]>
100
<Response [200]>
100
<Response [200]>
100
<Response [200]>
100
<Response [200]>
100
<Response [200]>
100
<Response [200]>
100
<Response [200]>
100
<Response [200]>
100
<Response [200]>
100
<Response [200]>
50
<Response [200]>
0


In [77]:
len(data.keys())

13

In [78]:
f = open("../data/vehicle_simulator_genre_158.pkl","wb")
pickle.dump(data,f)
f.close()


In [24]:
len(data["games12"])

50

In [2]:
data.keys()

NameError: name 'data' is not defined

### create a function that takes the data variable and outputs everything into a dataframe

In [79]:
with open('../data/vehicle_simulator_genre_158.pkl', 'rb') as f:
    data = pickle.load(f)


In [38]:
# max_genres =[]

In [62]:
# for key, values_list in data.items():
#     for vals_dicts in values_list:
#         for headers, values in vals_dicts.items():
#             if headers == "genres":
#                 num_genres = len(values)
#                 print(values[0]["genre_name"])
# #                 max_genres.append(num_genres)
            

Racing / driving
Racing / driving
Racing / driving
Racing / driving
Racing / driving
Simulation
Racing / driving
Racing / driving
Racing / driving
Racing / driving
Simulation
Simulation
Sports
Racing / driving
Simulation
Simulation
Simulation
Simulation
Simulation
Simulation
Sports
Simulation
Racing / driving
Simulation
Racing / driving
Simulation
Simulation
Action
Simulation
Racing / driving
Simulation
Racing / driving
Racing / driving
Simulation
Action
Simulation
Racing / driving
Simulation
Racing / driving
Racing / driving
Racing / driving
Simulation
Racing / driving
Simulation
Simulation
Racing / driving
Simulation
Simulation
Simulation
Simulation
Racing / driving
Racing / driving
Simulation
Racing / driving
Simulation
Racing / driving
Racing / driving
Racing / driving
Racing / driving
Racing / driving
Racing / driving
Racing / driving
Racing / driving
Racing / driving
Racing / driving
Action
Simulation
Racing / driving
Simulation
Racing / driving
Racing / driving
Racing / driving


In [ ]:
# max_genres.sort()
# print(max_genres)

In [80]:
title = []
game_id = []
description = []
moby_url = []
official_url = []
# the big list for platform lists
platform_primary = []
#initializing all the  lists we need
release_date1=[]
platform_id1=[]
platform_name1=[]
release_date2=[]
platform_id2=[]
platform_name2=[]
release_date3=[]
platform_id3=[]
platform_name3=[]
release_date4=[]
platform_id4=[]
platform_name4=[]
release_date5=[]
platform_id5=[]
platform_name5=[]
release_date6=[]
platform_id6=[]
platform_name6=[]
release_date7=[]
platform_id7=[]
platform_name7=[]
release_date8=[]
platform_id8=[]
platform_name8=[]
release_date9=[]
platform_id9=[]
platform_name9=[]
release_date10=[]
platform_id10=[]
platform_name10=[]
release_date11=[]
platform_id11=[]
platform_name11=[]
release_date12=[]
platform_id12=[]
platform_name12=[]
# release_date13=[]
# platform_id13=[]
# platform_name13=[]
# release_date14=[]
# platform_id14=[]
# platform_name14=[]
# release_date15=[]
# platform_id15=[]
# platform_name15=[]
#create lists to contain info for each platform
platform_secondary1=[]
platform_secondary2=[]
platform_secondary3=[]
platform_secondary4=[]
platform_secondary5=[]
platform_secondary6=[]
platform_secondary7=[]
platform_secondary8=[]
platform_secondary9=[]
platform_secondary10=[]
platform_secondary11=[]
platform_secondary12=[]
# platform_secondary13=[]
# platform_secondary14=[]
# platform_secondary15=[]

#append each of 3 data points to secondaries
platform_secondary1.append(release_date1)
platform_secondary1.append(platform_id1)
platform_secondary1.append(platform_name1)
platform_secondary2.append(release_date2)
platform_secondary2.append(platform_id2)
platform_secondary2.append(platform_name2)
platform_secondary3.append(release_date3)
platform_secondary3.append(platform_id3)
platform_secondary3.append(platform_name3)
platform_secondary4.append(release_date4)
platform_secondary4.append(platform_id4)
platform_secondary4.append(platform_name4)
platform_secondary5.append(release_date5)
platform_secondary5.append(platform_id5)
platform_secondary5.append(platform_name5)
platform_secondary6.append(release_date6)
platform_secondary6.append(platform_id6)
platform_secondary6.append(platform_name6)
platform_secondary7.append(release_date7)
platform_secondary7.append(platform_id7)
platform_secondary7.append(platform_name7)
platform_secondary8.append(release_date8)
platform_secondary8.append(platform_id8)
platform_secondary8.append(platform_name8)
platform_secondary9.append(release_date9)
platform_secondary9.append(platform_id9)
platform_secondary9.append(platform_name9)
platform_secondary10.append(release_date10)
platform_secondary10.append(platform_id10)
platform_secondary10.append(platform_name10)
platform_secondary11.append(release_date11)
platform_secondary11.append(platform_id11)
platform_secondary11.append(platform_name11)
platform_secondary12.append(release_date12)
platform_secondary12.append(platform_id12)
platform_secondary12.append(platform_name12)

#appending secondary lists to main platform list
platform_primary.append(platform_secondary1)
platform_primary.append(platform_secondary2)
platform_primary.append(platform_secondary3)
platform_primary.append(platform_secondary4)
platform_primary.append(platform_secondary5)
platform_primary.append(platform_secondary6)
platform_primary.append(platform_secondary7)
platform_primary.append(platform_secondary8)
platform_primary.append(platform_secondary9)
platform_primary.append(platform_secondary10)
platform_primary.append(platform_secondary11)
platform_primary.append(platform_secondary12)
# platform_primary.append(platform_secondary13)
# platform_primary.append(platform_secondary14)
# platform_primary.append(platform_secondary15)

In [81]:
### for genres
# the big list for platform lists
genre_primary=[]
# secondary lists for the max of 25 possible genres
genre_secondary1=[]
genre_secondary2=[]
genre_secondary3=[]
genre_secondary4=[]
genre_secondary5=[]
genre_secondary6=[]
genre_secondary7=[]
genre_secondary8=[]
genre_secondary9=[]
genre_secondary10=[]
genre_secondary11=[]
genre_secondary12=[]
genre_secondary13=[]
genre_secondary14=[]
genre_secondary15=[]
genre_secondary16=[]
genre_secondary17=[]
genre_secondary18=[]
genre_secondary19=[]
genre_secondary20=[]
genre_secondary21=[]
genre_secondary22=[]
genre_secondary23=[]
genre_secondary24=[]
genre_secondary25=[]

#genre_id and genre_name for each possible genre
genre_id1=[]
genre_name1=[]
genre_id2=[]
genre_name2=[]
genre_id3=[]
genre_name3=[]
genre_id4=[]
genre_name4=[]
genre_id5=[]
genre_name5=[]
genre_id6=[]
genre_name6=[]
genre_id7=[]
genre_name7=[]
genre_id8=[]
genre_name8=[]
genre_id9=[]
genre_name9=[]
genre_id10=[]
genre_name10=[]
genre_id11=[]
genre_name11=[]
genre_id12=[]
genre_name12=[]
genre_id13=[]
genre_name13=[]
genre_id14=[]
genre_name14=[]
genre_id15=[]
genre_name15=[]
genre_id16=[]
genre_name16=[]
genre_id17=[]
genre_name17=[]
genre_id18=[]
genre_name18=[]
genre_id19=[]
genre_name19=[]
genre_id20=[]
genre_name20=[]
genre_id21=[]
genre_name21=[]
genre_id22=[]
genre_name22=[]
genre_id23=[]
genre_name23=[]
genre_id24=[]
genre_name24=[]
genre_id25=[]
genre_name25=[]

#append names and genres to secondaries
genre_secondary1.append(genre_id1)
genre_secondary1.append(genre_name1)
genre_secondary2.append(genre_id2)
genre_secondary2.append(genre_name2)
genre_secondary3.append(genre_id3)
genre_secondary3.append(genre_name3)
genre_secondary4.append(genre_id4)
genre_secondary4.append(genre_name4)
genre_secondary5.append(genre_id5)
genre_secondary5.append(genre_name5)
genre_secondary6.append(genre_id6)
genre_secondary6.append(genre_name6)
genre_secondary7.append(genre_id7)
genre_secondary7.append(genre_name7)
genre_secondary8.append(genre_id8)
genre_secondary8.append(genre_name8)
genre_secondary9.append(genre_id9)
genre_secondary9.append(genre_name9)
genre_secondary10.append(genre_id10)
genre_secondary10.append(genre_name10)
genre_secondary11.append(genre_id11)
genre_secondary11.append(genre_name11)
genre_secondary12.append(genre_id12)
genre_secondary12.append(genre_name12)
genre_secondary13.append(genre_id13)
genre_secondary13.append(genre_name13)
genre_secondary14.append(genre_id14)
genre_secondary14.append(genre_name14)
genre_secondary15.append(genre_id15)
genre_secondary15.append(genre_name15)
genre_secondary16.append(genre_id16)
genre_secondary16.append(genre_name16)
genre_secondary17.append(genre_id17)
genre_secondary17.append(genre_name17)
genre_secondary18.append(genre_id18)
genre_secondary18.append(genre_name18)
genre_secondary19.append(genre_id19)
genre_secondary19.append(genre_name19)
genre_secondary20.append(genre_id20)
genre_secondary20.append(genre_name20)
genre_secondary21.append(genre_id21)
genre_secondary21.append(genre_name21)
genre_secondary22.append(genre_id22)
genre_secondary22.append(genre_name22)
genre_secondary23.append(genre_id23)
genre_secondary23.append(genre_name23)
genre_secondary24.append(genre_id24)
genre_secondary24.append(genre_name24)
genre_secondary25.append(genre_id25)
genre_secondary25.append(genre_name25)

#append secondaries to genre_primary
genre_primary.append(genre_secondary1)
genre_primary.append(genre_secondary2)
genre_primary.append(genre_secondary3)
genre_primary.append(genre_secondary4)
genre_primary.append(genre_secondary5)
genre_primary.append(genre_secondary6)
genre_primary.append(genre_secondary7)
genre_primary.append(genre_secondary8)
genre_primary.append(genre_secondary9)
genre_primary.append(genre_secondary10)
genre_primary.append(genre_secondary11)
genre_primary.append(genre_secondary12)
genre_primary.append(genre_secondary13)
genre_primary.append(genre_secondary14)
genre_primary.append(genre_secondary15)
genre_primary.append(genre_secondary16)
genre_primary.append(genre_secondary17)
genre_primary.append(genre_secondary18)
genre_primary.append(genre_secondary19)
genre_primary.append(genre_secondary20)
genre_primary.append(genre_secondary21)
genre_primary.append(genre_secondary22)
genre_primary.append(genre_secondary23)
genre_primary.append(genre_secondary24)
genre_primary.append(genre_secondary25)


In [82]:
len(genre_primary)

25

In [83]:
for key, values_list in data.items():
    for vals_dicts in values_list:
        for headers, values in vals_dicts.items():
            if headers == "platforms":
                range_num = list(range(0,12))
                for num in range_num:
                    try:
                        platform_primary[num][0].append(values[num]["first_release_date"])
                        platform_primary[num][1].append(values[num]["platform_id"])
                        platform_primary[num][2].append(values[num]["platform_name"])
                    except:
                        platform_primary[num][0].append("")
                        platform_primary[num][1].append("")
                        platform_primary[num][2].append("")
            elif headers == "genres":
                g_range_num = list(range(0,25))
                for g_num in g_range_num:
                    try:
                        genre_primary[g_num][0].append(values[g_num]["genre_id"])
                        genre_primary[g_num][1].append(values[g_num]["genre_name"])
                        print("try")
                    except:
                        genre_primary[g_num][0].append("")
                        genre_primary[g_num][1].append("")
                        print("except")
            elif headers == "description":
                description.append(values)
            elif headers == "title":
                title.append(values)
            elif headers == "moby_url":
                moby_url.append(values)
            elif headers == "game_id":
                game_id.append(values)
            elif headers == "official_url":
                official_url.append(values)

try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
excep

try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except

except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except


except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
tr

except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
exce

...

except
except
except
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
except
except
except
except
except
except
except


except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
except
except
exc

try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
ex

except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except

try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
try
try
try
try
try
try
try
try
try
except
except
except
except
except
except
except
except
except
except
except
except
except
e

In [69]:
genre_id1

[6,
 6,
 6,
 6,
 6,
 3,
 6,
 6,
 6,
 6,
 3,
 3,
 5,
 6,
 3,
 3,
 3,
 3,
 3,
 3,
 5,
 3,
 6,
 3,
 6,
 3,
 3,
 1,
 3,
 6,
 3,
 6,
 6,
 3,
 1,
 3,
 6,
 3,
 6,
 6,
 6,
 3,
 6,
 3,
 3,
 6,
 3,
 3,
 3,
 3,
 6,
 6,
 3,
 6,
 3,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 1,
 3,
 6,
 3,
 6,
 6,
 6,
 3,
 3,
 3,
 6,
 6,
 6,
 3,
 6,
 6,
 6,
 1,
 6,
 6,
 1,
 3,
 3,
 3,
 3,
 1,
 5,
 6,
 3,
 3,
 6,
 3,
 6,
 6,
 3,
 3,
 3,
 6,
 3,
 6,
 3,
 6,
 6,
 6,
 6,
 3,
 3,
 3,
 3,
 6,
 6,
 6,
 6,
 3,
 6,
 6,
 3,
 3,
 6,
 3,
 3,
 3,
 1,
 6,
 6,
 6,
 3,
 1,
 3,
 6,
 6,
 6,
 3,
 6,
 3,
 3,
 3,
 3,
 6,
 3,
 6,
 6,
 6,
 6,
 5,
 6,
 3,
 6,
 6,
 5,
 6,
 3,
 3,
 6,
 3,
 6,
 6,
 6,
 3,
 3,
 6,
 6,
 1,
 6,
 3,
 6,
 3,
 3,
 3,
 6,
 3,
 6,
 6,
 3,
 1,
 3,
 3,
 3,
 3,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 6,
 6,
 3,
 6,
 3,
 6,
 3,
 6,
 6,
 3,
 6,
 3,
 6,
 3,
 6,
 3,
 6,
 6,
 6,
 6,
 3,
 6,
 3,
 6,
 3,
 1,
 3,
 6,
 3,
 6,
 6,
 3,
 6,
 6,
 3,
 3,
 6,
 1,
 3,
 6,
 6,
 6,
 6,
 6,
 6,
 3,
 3,
 3,
 6,
 3,
 6,
 6,
 3,
 3,
 6,
 6,
 1,
 3,
 6,


In [84]:
vehicle_sim_158 = pd.DataFrame()

In [85]:
# ok, we got the stuff parsed correctly for platforms, and it shoulld append nicely to the main list.
# just need to append correctly with correct slicing, and add column names goodly

main_list = []
main_list.append(title)
main_list.append(game_id)
main_list.append(description)
main_list.append(moby_url)
main_list.append(official_url)
#you can use a for loop or 2 to automate the appending of lists to main list - just nest 'em, me thinks
for secondaries in platform_primary:
    for datalists in secondaries:
        main_list.append(datalists)
for secondaries_g in genre_primary:
    for datalists_g in secondaries_g:
        main_list.append(datalists_g)

In [86]:
vehicle_sim_158 = pd.DataFrame(main_list).transpose()
vehicle_sim_158.columns = ["title", "game_id", "description", "moby_url", "official_url", "release_date1",
"platform_id1",
"platform_name1",
"release_date2",
"platform_id2",
"platform_name2",
"release_date3",
"platform_id3",
"platform_name3",
"release_date4",
"platform_id4",
"platform_name4",
"release_date5",
"platform_id5",
"platform_name5",
"release_date6",
"platform_id6",
"platform_name6",
"release_date7",
"platform_id7",
"platform_name7",
"release_date8",
"platform_id8",
"platform_name8",
"release_date9",
"platform_id9",
"platform_name9",
"release_date10",
"platform_id10",
"platform_name10",
"release_date11",
"platform_id11",
"platform_name11",
"release_date12",
"platform_id12",
"platform_name12",
"genre_id1",
"genre_name1",
"genre_id2",
"genre_name2",
"genre_id3",
"genre_name3",
"genre_id4",
"genre_name4",
"genre_id5",
"genre_name5",
"genre_id6",
"genre_name6",
"genre_id7",
"genre_name7",
"genre_id8",
"genre_name8",
"genre_id9",
"genre_name9",
"genre_id10",
"genre_name10",
"genre_id11",
"genre_name11",
"genre_id12",
"genre_name12",
"genre_id13",
"genre_name13",
"genre_id14",
"genre_name14",
"genre_id15",
"genre_name15",
"genre_id16",
"genre_name16",
"genre_id17",
"genre_name17",
"genre_id18",
"genre_name18",
"genre_id19",
"genre_name19",
"genre_id20",
"genre_name20",
"genre_id21",
"genre_name21",
"genre_id22",
"genre_name22",
"genre_id23",
"genre_name23",
"genre_id24",
"genre_name24",
"genre_id25",
"genre_name25"]

In [87]:
vehicle_sim_158

,title,game_id,description,moby_url,official_url,release_date1,platform_id1,platform_name1,release_date2,platform_id2,...,genre_id21,genre_name21,genre_id22,genre_name22,genre_id23,genre_name23,genre_id24,genre_name24,genre_id25,genre_name25
0,Indianapolis 500: The Simulation,5,"The famous Indianapolis 500 Mile race, held an...",http://www.mobygames.com/game/indianapolis-500...,None,1989-12,2,DOS,1990,19,...,,,,,,,,,,
1,Test Drive,107,This mix of racing simulation and arcade game ...,http://www.mobygames.com/game/test-drive,None,1987,2,DOS,1987,27,...,,,,,,,,,,
2,Moto Racer,127,An arcade motorcycle racer in which both super...,http://www.mobygames.com/game/moto-racer,None,1997,3,Windows,1997-11,6,...,,,,,,,,,,
3,Test Drive III: The Passion,158,Drive exotic cars while fleeing police in this...,http://www.mobygames.com/game/test-drive-iii-t...,None,1990,2,DOS,,,...,,,,,,,,,,
4,Star Wars: Episode I - Racer,276,Based upon the Pod Racing scenes of Star Wars:...,http://www.mobygames.com/game/star-wars-episod...,None,1999,3,Windows,1999-05-18,9,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,NASCAR Challenge,173715,None,http://www.mobygames.com/game/nascar-challenge,None,1999,11,Game Boy Color,,,...,,,,,,,,,,
1246,NASCAR Heat: Ultimate Edition+,175740,None,http://www.mobygames.com/game/nascar-heat-ulti...,None,2021-11-19,203,Nintendo Switch,,,...,,,,,,,,,,
1247,Farming Simulator 22,175746,None,http://www.mobygames.com/game/farming-simulato...,https://farming-simulator.com,2021-11-21,142,Xbox One,2021-11-21,289,...,,,,,,,,,,
1248,Crash City: Heavy Traffic Drive,175750,None,http://www.mobygames.com/game/crash-city-heavy...,None,2017-07-07,86,iPhone,2017-07-07,96,...,,,,,,,,,,


In [89]:
vehicle_sim_158.to_csv("../data/vehicle_sim_158_df_w_genre.csv")

### next steps
* create new notebook for all these steps below
* try to clean this up - see if certian genre categories don't belong in this data set
* check for duplicates? - DONE
* convert all release date columns to date time, compare them with each other in order to create new columns with earliest release date, earliest platform, earliest platform id
* try to add company data by using the process described here: https://www.mobygames.com/forums/dga,2/dgb,13/dgm,244369/
* once you have company data for earliest release, figure out a way to use it to scrape wikipedia or some other website in order to get city and state of headquarters, in order to allow for geographical filtering
